In [1]:
%%capture
import contrails_utils as cu

In [2]:
print(cu.Commun.colors.GREEN.format("contrails_utils imported"))

contrails_utils imported


In [3]:
# %%capture
import os

import numpy as np
import pandas as pd

import yaml
from pathlib import Path

import torch

In [4]:
### Kaggle environment variable
INTERACTIVE = os.getenv("KAGGLE_KERNEL_RUN_TYPE") == "Interactive"
RERUN = os.getenv("KAGGLE_IS_COMPETITION_RERUN") is not None
print(f"Interactive {INTERACTIVE}, RERUN {RERUN}")

Interactive False, RERUN False


In [5]:
# # # # MODEL_PATH = "/kaggle/working/models/"
# CFG_PATH = Path("/kaggle/input/training-with-4-folds")
# # CFG_PATH = Path("/kaggle/input/identify-contrails-models/models_kaggle_contrails")
# with open(CFG_PATH/ "config.yaml", "r") as file_obj:
#     config = yaml.safe_load(file_obj)

In [6]:
config = cu.PyLModel.CFG.default_predict_config(image_size=384)
# or config = PyLModel.CFG.open_config(CFG_PATH)

In [7]:
N_TIMES_BEFORE = 4

data_path = Path('/kaggle/input/google-research-identify-contrails-reduce-global-warming')
data_test = data_path / 'test'
submission = pd.read_csv(data_path /'sample_submission.csv', index_col='record_id')

In [8]:
test_df = pd.DataFrame([{'record_id': path_test.name, 'path':path_test, "sequence_num":N_TIMES_BEFORE} for path_test in data_test.iterdir()])
# test_df

# Submission 

In [9]:
%%time
# paths_model:list[Path] = [
#     Path("/kaggle/input/identify-contrails-models/model-seq0-val_dice0.6326.ckpt"),
#     Path("/kaggle/input/pseudolabel-train/models/model-seq1-val_dice=0.6419.ckpt"),
#     Path("/kaggle/input/identify-contrails-models/models_kaggle_contrails/pb_model_dice_649.ckpt"),
# ] +list(Path("/kaggle/input/training-with-4-folds/models").glob("*.ckpt"))

# paths_model = list(Path("/kaggle/input/training-with-4-folds/models").glob("*0.66*.ckpt")) #[Path("/kaggle/input/training-with-4-folds/models/model-f0-val_dice=0.6616.ckpt")]

# paths_model = list(Path("/kaggle/input/training-with-4-folds/models").glob("*.ckpt"))

paths_model = list(Path("/kaggle/input/pseudolabel-train/models").glob("*.ckpt"))
# threshold = 0.15 
high_threshold, low_threshold = (0.355, 0.31)
all_preds, models_len = cu.PyLModel.predict_with_models(test_df, models=paths_model, config=config)

Predict with model: model-from-scratch-v2-real-train-val_dice=0.6013.ckpt
CPU times: user 4.08 s, sys: 1.86 s, total: 5.94 s
Wall time: 13 s


In [10]:
%%time
for index in submission.index.tolist():
    predicted_mask_with_threshold = cu.PyLModel.stack_prediction_mask_threshold(all_preds, index, high_threshold, low_threshold, models_len, vote_majority=0.45)
    submission.loc[int(index), 'encoded_pixels'] = cu.Commun.run_length_encode(predicted_mask_with_threshold)

CPU times: user 6.92 ms, sys: 753 µs, total: 7.67 ms
Wall time: 11.5 ms


/kaggle/usr/lib/contrails_utils/contrails_utils.py:115: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/native/TensorShape.cpp:3571.)
  dots = np.where((x.T.flatten() == fg_val_min))[0]


In [11]:
submission.to_csv('submission.csv')
submission

,encoded_pixels
record_id,
1000834164244036115,27842 1 28098 1 28353 2 28608 3 28864 3 29119 ...
1002653297254493116,-


In [12]:
# [pred for pred in preds for preds in all_preds["f0"].values()]